# Test #1

## Nom à trouver

In this test, pyKasso is used on a simple rectangular domain, with one inlet in the center of the upper left limit of the model, and one outlet in the center of the bottom right limit of the model.
pyKasso is firstly used with the isotropic fast-marching algorithm then with the anisotropic one.  

In [1]:
### Load pyKasso and other packages
import copy
import numpy as np
import pykasso as pk
import pykasso.tools as pkt
import pykasso.visualization as pkv

In [2]:
pk.create_project('geometry_01')
path_project = 'geometry_01/outputs/project_state.yaml'

CAUTION: You are using the development version of this package.


In [3]:
sks_settings = {
    'sks' : {
        'seed' : 1111
    },
    'grid' : {
        'x0' : 0,
        'y0' : 0,
        'z0' : 100, # z0 interdit en-dessous de 0
        'nx' : 100,
        'ny' : 50,
        'nz' : 50,
        'dx' : 10,
        'dy' : 10,
        'dz' : 10
    },
    'outlets' : {
        'number'     : 1,
        'data'       : [[994, 250]],
        'importance' : [1],
        'mode'       : 'domain_bottom',
    },
    'inlets' : {
        'number'     : 1,
        'data'       : [[0, 250]],
        'per_outlet' : [1],
        'importance' : [1],
        'mode'       : 'domain_surface',
    },
}

### Cas simple

In [4]:
for algorithm in ['Isotropic3', 'Riemann3']:
    sks_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=sks_settings)
    sim.build_model()
    sim.compute_karst_network()

In [5]:
visualization = pkv.Visualization(path_project)
settings = {
    'ghosts'  : [0],
    'outline' : True,
    'inlets' : True,
    'outlets' : True,
    'cpos' : 'xz',
}
visualization.show([0,1], ['karst'], settings=settings)

## Influence de la faille

In [6]:
# Constructs the faults
faults = np.full((100,50,50), 0)
faults[:, :, 25] = 1

# Visualizes the faults
# pkv.show_array(faults)

# Updates the settings
new_settings = copy.deepcopy(sks_settings)
new_settings['faults'] = {'data' : faults}

In [7]:
for algorithm in ['Isotropic3', 'Riemann3']:
    new_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=new_settings)
    sim.build_model()
    sim.compute_karst_network()

In [8]:
visualization = pkv.Visualization(path_project)
# visualization.show([2,3], ['karst', 'faults'], settings=settings)

## Influence de la fracturation

In [9]:
# Declares the fractures settings
fractures = {
    'seed': 111111,
    'settings': {
        'family_01': {
            'alpha'        : 2,
            'density'      : 0.00001,
            'orientation'  : 180,
            'dip'          : [10,20],
            'length'       : 500,
        }, 
        'family_02': {
            'alpha'        : 2,
            'density'      : 0.00001,
            'orientation'  : 360,
            'dip'          : 90,
            'length'       : 500,
        }
    }
}

# Updates the settings
new_settings = copy.deepcopy(sks_settings)
new_settings['fractures'] = fractures

In [10]:
for algorithm in ['Isotropic3', 'Riemann3']:
    new_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=new_settings)
    sim.build_model()
    sim.compute_karst_network()

error
error
error
error


In [11]:
# error ?

In [12]:
visualization = pkv.Visualization(path_project)
# visualization.show([4,5], ['karst', 'fractures'], settings=settings)

## Influence de la geologie

In [13]:
# Constructs the geology
geology = np.full((100,50,50), 1)
geology[:, 35:50, :] = 2

# Visualizes the geology
# pkv.show_array(geology)

# Updates the settings
new_settings = copy.deepcopy(sks_settings)
new_settings['geology'] = {
    'data' : geology,
    'costs' : {1: 0.6, 2: 0.4}
}

In [14]:
for algorithm in ['Isotropic3', 'Riemann3']:
    new_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=new_settings)
    sim.build_model()
    sim.compute_karst_network()

In [16]:
visualization = pkv.Visualization(path_project)
# visualization.show([6,7], ['karst', 'cost'], settings=settings)